In [12]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama
import json
import re
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath("MHQA-ontology"))


In [13]:
from llama_index.llms.ollama import Ollama

llm = Ollama(model="llama3.1:70b", request_timeout= 240.0)

# response = llm.complete("What are the recurring archetypes in Bulgarian political myths?")
# print(response)

In [14]:
llm.complete("What are the recurring archetypes in Bulgarian political myths?")

CompletionResponse(text="Bulgarian political mythology is rich and diverse, reflecting the country's complex history, cultural heritage, and national identity. While it's challenging to provide an exhaustive list, I'll outline some recurring archetypes that appear in Bulgarian political myths:\n\n1. **The National Hero**: Figures like Vasil Levski, Hristo Botev, and Georgi Rakovski are revered as symbols of national resistance against Ottoman rule. They embody the values of patriotism, bravery, and self-sacrifice.\n2. **The Wise Leader**: Archetypes like Khan Krum, Tsar Simeon I, and Prince Alexander Batenberg represent wise and just rulers who have guided Bulgaria through times of turmoil and transformation.\n3. **The Martyr for Freedom**: This archetype is exemplified by figures like Stefan Stambolov, a politician who was assassinated in 1895, and Nikola Vaptsarov, a poet who was executed by the Communist authorities in 1942. They are celebrated as champions of freedom and democracy.

In [15]:

from llama_index.core import PromptTemplate

Hotpot_train = json.load(open('MHQA-ontology/dataset/hotpot/hotpot_train_v1.1.json'))
# Hotpot_train = json.load(open('MHQA-ontology/dataset/hotpot/hotpot_train_v1.1.json'))

In [16]:
query_data = Hotpot_train[4]

In [17]:
query_data


{'supporting_facts': [['Cadmium chloride', 1], ['Ethanol', 0]],
 'level': 'medium',
 'question': 'Cadmium Chloride is slightly soluble in this chemical, it is also called what?',
 'context': [['Cadmium chloride',
   ['Cadmium chloride is a white crystalline compound of cadmium and chlorine, with the formula CdCl.',
    ' It is a hygroscopic solid that is highly soluble in water and slightly soluble in alcohol.',
    ' Although it is considered to be ionic, it has considerable covalent character to its bonding.',
    ' The crystal structure of cadmium chloride (described below), composed of two-dimensional layers of ions, is a reference for describing other crystal structures.',
    ' Also known are CdCl•HO and CdCl•5HO.']],
  ['Water blue',
   ['Water blue, also known as aniline blue, Acid blue 22, Soluble Blue 3M, Marine Blue V, or C.I. 42755, is a chemical compound used as a stain in histology.',
    ' Water blue stains collagen blue in tissue sections.',
    ' It is soluble in water

In [18]:
query_data['supporting_facts']


[['Cadmium chloride', 1], ['Ethanol', 0]]

In [19]:
import json
import re

def extract_json(response_text: str) -> dict:
    """
    Extract JSON data from text response using multiple patterns and approaches.
    
    Args:
        response_text (str): The raw text response containing JSON data
        
    Returns:
        dict: Parsed JSON data, or None if no valid JSON found
        
    Example:
        >>> text = '''Here's the response:
        ... ```
        ... {
        ...   "triples": []
        ... }
        ... ```'''
        >>> result = extract_json_from_response(text)
    """
    text = str(response_text)
    
    # Try code block pattern first
    code_block_match = re.search(r'```(?:json)?\s*(\{[\s\S]*?\})\s*```', text)
    if code_block_match:
        try:
            return json.loads(code_block_match.group(1))
        except json.JSONDecodeError:
            pass

    # Patterns for both Python dict and JSON formats
    patterns = [
        r'\{\'[^\']+\'\s*:\s*\'[^\']+\'\s*,\s*\'[^\']+\'\s*:\s*\[.*?\]\}',  # Python dict pattern
        r'\{\s*"[^"]+"\s*:\s*"[^"]+"\s*,\s*"[^"]+"\s*:\s*\[.*?\]\s*\}',    # JSON pattern
        r'\{[\s\S]*?\}'  # Fallback general JSON object pattern
    ]
    
    for pattern in patterns:
        match = re.search(pattern, text, re.MULTILINE | re.DOTALL)
        if match:
            json_string = match.group().strip()
            # Replace single quotes with double quotes for Python dict format
            json_string = json_string.replace("'", '"')
            try:
                return json.loads(json_string)
            except json.JSONDecodeError:
                continue
    
    return None
# def extract_json(response):
#     text = str(response)
    
#     # First pattern to match Python dictionary-like structure
#     pattern1 = r'\{\'[^\']+\'\s*:\s*\'[^\']+\'\s*,\s*\'[^\']+\'\s*:\s*\[.*?\]\}'
    
#     # Second pattern for standard JSON structure
#     pattern2 = r'\{\s*"[^"]+"\s*:\s*"[^"]+"\s*,\s*"[^"]+"\s*:\s*\[.*?\]\s*\}'
    
#     # Try both patterns
#     dict_match = re.search(pattern1, text, re.MULTILINE | re.DOTALL)
#     json_match = re.search(pattern2, text, re.MULTILINE | re.DOTALL)
    
#     if dict_match:
#         # Handle Python dictionary format
#         dict_string = dict_match.group().strip()
#         # Replace single quotes with double quotes
#         json_string = dict_string.replace("'", '"')
#         try:
#             return json.loads(json_string)
#         except json.JSONDecodeError as e:
#             print(f"Error parsing dictionary format: {e}")
    
#     if json_match:
#         # Handle JSON format
#         json_string = json_match.group().strip()
#         try:
#             return json.loads(json_string)
#         except json.JSONDecodeError as e:
#             print(f"Error parsing JSON format: {e}")
    
#     return None

Here is the evaluation part, we still not doing this for now

In [20]:
def evaluation(hotpot, llm):
    template = (
        "We have provided context information below. \n"
        "---------------------\n"
        "{context_str}"
        "\n---------------------\n"
        "Given this information, please answer the question: {query_str}\n"
        "Also list all the support fact sentences from the context that you used to answer the question."
        "Since the context information is the list format, please give the support fact sentence.\n"
        """For example, if the question is:
        
        What nationality was James Henry Miller's wife?
        
        And you find the answer is American, and used the following support fact sentences:
        
        ['Peggy Seeger',
        ['Margaret "Peggy" Seeger (born June 17, 1935) is an American folksinger.',
            'She is also well known in Britain, where she has lived for more than 30 years, and was married to the singer and songwriter Ewan MacColl until his death in 1989.']],
        ['Ewan MacColl',
        ['James Henry Miller (25 January 1915 – 22 October 1989), better known by his stage name Ewan MacColl, was an English folk singer, songwriter, communist, labour activist, actor, poet, playwright and record producer.']],

        You should give me explanation and also give me support fact's index by: [['Peggy Seeger', 0], ['Peggy Seeger', 1], ['Ewan MacColl', 0]].
        Please only help me output the index of supporting fact but not the content of the supporting fact.

        Please do the following step by step:

        1. Output by JSON format:

        {"Answer":
            "American"
        ,
        "Supporting fact":
            [["Peggy Seeger", 0], ["Peggy Seeger", 1], ["Ewan MacColl", 0]]}

        2. Double check if the supporting fact you output satisfied I required. Note that I only want you to output the index of the supporting fact but not the content of the supporting fact,
        in this kinds of format [["Peggy Seeger", 0], ["Peggy Seeger", 1], ["Ewan MacColl", 0]].

        3. Double check again if the supporting fact you output satisfied I required.

        """
    )
    qa_template = PromptTemplate(template)

    # you can create text prompt (for completion API)

    result = pd.DataFrame(columns=['question', 'gold_answer', 'gold_supporting_facts', 'llm_answer', 'llm_supporting_facts','llm_answer_correctness', 'llm_supporting_facts_correctness'])


    for query_data in hotpot:
        retry_count = 0
        while True:
            # Extract gold answer and supporting facts from the given dataset
            question = query_data['question']
            gold_answer = query_data['answer']
            gold_supporting_facts = query_data['supporting_facts']

            prompt = qa_template.format(context_str=query_data['context'], query_str=query_data['question'])

            response = llm.complete(prompt)
            # Generate answer by llm 
            json_response = extract_json(str(response))

            # check if it contains json format response

            if type(json_response) == dict:

                answer = json_response['Answer']
                supporting_facts = json_response['Supporting fact']
                
                answer_matched = False
                if answer.lower() == gold_answer.lower() or answer.lower() in gold_answer.lower() or gold_answer.lower() in answer.lower():
                    answer_matched = True

                Evidence_matched = False
                


                # Check if the generated answer and supporting fact is correct or not, and add a row in the result dataframe
                if answer_matched is True and supporting_facts == gold_supporting_facts:
                    
                    new_row = pd.DataFrame({
                        'question': [question],
                        'gold_answer': [gold_answer],
                        'gold_supporting_facts': [gold_supporting_facts],
                        'llm_answer': [answer],
                        'llm_supporting_facts': [supporting_facts],
                        'llm_answer_correctness': True,
                        'llm_supporting_facts_correctness': True
                    })

                    result = pd.concat([result,new_row], ignore_index=True)
                    break
                elif answer_matched is True and supporting_facts != gold_supporting_facts:
                    new_row = pd.DataFrame({
                        'question': [question],
                        'gold_answer': [gold_answer],
                        'gold_supporting_facts': [gold_supporting_facts],
                        'llm_answer': [answer],
                        'llm_supporting_facts': [supporting_facts],
                        'llm_answer_correctness': True,
                        'llm_supporting_facts_correctness': False
                    })

                    result = pd.concat([result,new_row], ignore_index=True)
                    break
                elif answer_matched is False and supporting_facts == gold_supporting_facts:
                    new_row = pd.DataFrame({
                        'question': [question],
                        'gold_answer': [gold_answer],
                        'gold_supporting_facts': [gold_supporting_facts],
                        'llm_answer': [answer],
                        'llm_supporting_facts': [supporting_facts],
                        'llm_answer_correctness': False,
                        'llm_supporting_facts_correctness': True
                    })

                    result = pd.concat([result,new_row], ignore_index=True)
                    break
                else:
                    new_row = pd.DataFrame({
                        'question': [question],
                        'gold_answer': [gold_answer],
                        'gold_supporting_facts': [gold_supporting_facts],
                        'llm_answer': [answer],
                        'llm_supporting_facts': [supporting_facts],
                        'llm_answer_correctness': False,
                        'llm_supporting_facts_correctness': False
                    })

                    result = pd.concat([result,new_row], ignore_index=True)
                    break

            else:
                if retry_count == 0:
                    retry_count += 1
                    print(f"Error: {question}, retrying {retry_count} times")
                    print('##############################################################################')
                    print(response)
                    print('##############################################################################')
                    answer = None
                    supporting_facts = None
                    continue
                else:
                    print(f"Error: {question},after retrying {retry_count} times")
                    print(response)
                    answer = None
                    supporting_facts = None
                    break

    return result


test_query_data = Hotpot_train[:50]
result_df = evaluation(test_query_data,llm)

Error: Which magazine was started first Arthur's Magazine or First for Women?, retrying 1 times
##############################################################################
{"Answer": "Arthur's Magazine", "Supporting fact": [["Arthur's Magazine", 0], ["First for Women", 2]]}
##############################################################################
Error: Which magazine was started first Arthur's Magazine or First for Women?,after retrying 1 times
{"Answer": "Arthur's Magazine", 
"Supporting fact": [["Arthur's Magazine", 0], ["First for Women", 2]]}
Error: Pauline Henry was known as the vocalist of a very popular cover song. Which album was this song from?, retrying 1 times
##############################################################################
{"Answer": "The Chimes (album)", "Supporting fact": [["Livin' in the Sunlight, Lovin' in the Moonlight", 1], ["Pauline Henry", 2], ["The Chimes (Scottish band)", 3]]}
#################################################################

,question,gold_answer,gold_supporting_facts,llm_answer,llm_supporting_facts,llm_answer_correctness,llm_supporting_facts_correctness
0,The Oberoi family is part of a hotel company t...,Delhi,"[[Oberoi family, 0], [The Oberoi Group, 0]]",Delhi,"[[The Oberoi Group, 0]]",True,False
1,Musician and satirist Allie Goertz wrote a son...,President Richard Nixon,"[[Allie Goertz, 0], [Allie Goertz, 1], [Allie ...",President Richard Nixon,"[[Milhouse Van Houten, 0]]",True,False
2,What nationality was James Henry Miller's wife?,American,"[[Peggy Seeger, 0], [Peggy Seeger, 1], [Ewan M...",American,"[[Peggy Seeger, 0], [Peggy Seeger, 1], [Ewan M...",True,True
3,Cadmium Chloride is slightly soluble in this c...,alcohol,"[[Cadmium chloride, 1], [Ethanol, 0]]",alcohol,"[[Cadmium chloride, 1], [Water blue, 2], [Difl...",True,False
4,Which tennis player won more Grand Slam titles...,Jonathan Stark,"[[Jonathan Stark (tennis), 0], [Jonathan Stark...",Jonathan Stark,"[[Henri Leconte, 2], [Jonathan Stark (tennis),...",True,False
5,Which genus of moth in the world's seventh-lar...,Crambidae,"[[Indogrammodes, 0], [Indogrammodes, 1], [Indi...",Indogrammodes,"[[India, 1], [Geography of India, 2], [List of...",False,False
6,Who was once considered the best kick boxer in...,Badr Hari,"[[Global Fighting Championship, 1], [Global Fi...",Badr Hari,"[[Badr Hari, 2]]",True,False
7,"The Dutch-Belgian television series that ""Hous...",2006,"[[House of Anubis, 0], [Het Huis Anubis, 1]]",2006,"[[Het Huis Anubis, 2]]",True,False
8,What is the length of the track where the 2013...,6.213 km long,"[[2013 Liqui Moly Bathurst 12 Hour, 0], [2013 ...",6.213 km,"[[Mount Panorama Circuit, 2]]",True,False
9,"Fast Cars, Danger, Fire and Knives includes gu...",Jaime Meline,"[[Fast Cars, Danger, Fire and Knives, 2], [El-...",El-P,"[[Fast Cars, Danger, Fire and Knives, 3]]",False,False


In [22]:
precision = result_df['llm_answer_correctness'].sum() / len(result_df)
recall = result_df['llm_supporting_facts_correctness'].sum() / len(result_df)

print(f"Precision: {precision}")
print(f"Recall: {recall}")

result_df.to_csv('result_raw_2500.csv', index=False)

Precision: 0.7708333333333334
Recall: 0.14583333333333334
